In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from stopping_power_ml.stop_distance import StoppingDistanceComputer
from stopping_power_ml.integrator import TrajectoryIntegrator
from sklearn.neighbors import KernelDensity
from scipy.interpolate import UnivariateSpline
from scipy.signal import savgol_filter
from scipy.optimize import fmin
import pickle as pkl
import pandas as pd
import numpy as np
import keras
import os
import glob
import statistics
import json
%load_ext autoreload
%autoreload 2

## Load model generated from previous training

In [ ]:
model = keras.models.load_model(os.path.join('./multiple-velocities', 'model-random-and-channel.h5'))

## Initialize Featurizers

In [ ]:
with open(os.path.join('./multiple-velocities', 'featurizer.pkl'), 'rb') as fp:
    featurizers = pkl.load(fp)

## Load Aluminum Lattice

In [ ]:
start_frame = pkl.load(open('al_starting_frame.pkl', 'rb'))

## Load Channel Data

In [ ]:
channel_data = pd.read_pickle(os.path.join('single-velocity', 'data', 'channel_data.pkl.gz')).query('average_range')

## Initialize Trajectory Integrator
This is the class object from the integrator.py file. Will be initialized with aluminum lattice, model and featurizers. Will be used to in intializing the StopDistanceComputer class object.

In [ ]:
traj_int = TrajectoryIntegrator(start_frame, model, featurizers)

## Initialize Stopping Distance Computer
Initializes StopDistanceComputer class object from stop_distance.py, which is used to call the stopping distance function.

The stopping distance function takes a starting point and direction and calculates the trajectory of a particle along that direction. The displacement and the velocity of the particle is output with respect to time. 

The third input parameter of the compute_stopping_distance function is a float value which multiplies the force on the particle as it traverse. The reason for this, is to see the particles behavior as the force is manually changed.

In [1]:
stop_comp = StoppingDistanceComputer(traj_int)

NameError: name 'StoppingDistanceComputer' is not defined

## Testing Stopping Distance
The stopping distance function has to be operated at a series of starting points, facing in the 111 direction. Thus the following function takes the x, y, z coordinates for the starting points and iterates through the stopping distance function for the combination of starting points. 

It can be seen that the compute_stopping_distance function has two main input parameters. The first is the starting point as a a list, where each element is the x, y and z point in order. The second input is a list showing the direction, normalized. It can be seen that the direction is 111, normalized. The combination of x, y, and z points are input as the starting coordinates, and the stopping distance function is run for each. The output is a json file which lists 

In [ ]:
def force_variation(x_coords,y_coords,z_coords,scalar,path):
    
    #y_coords is array of y positions
    #z_coords is array of z positions
    if x_coords == 0: 
        for i in y_coords:
            for j in z_coords:
                dist_channel,pos_channel = stop_comp.compute_stopping_distance([0,i,j],[4/np.sqrt(3), 4/np.sqrt(3), 4/np.sqrt(3)],scalar,output=1)
                #dist_channel,pos_channel = stop_comp.compute_stopping_distance([0,i,j],[4, 0, 0],scalar,output=1)
                with open(os.path.join(path,f"traj_{i}_{j}.json"), "w") as f:
                    f.write(pos_channel.to_json(orient="columns", lines=False))
    else:
        for i in x_coords:
            for j in y_coords:
                for k in z_coords:
                    dist_channel,pos_channel = stop_comp.compute_stopping_distance([i,j,k],[4/np.sqrt(3), 4/np.sqrt(3), 4/np.sqrt(3)],scalar,output=1)
                    with open(os.path.join(path,f"traj_{i}_{j}_{k}.json"), "w") as f:
                        f.write(pos_channel.to_json(orient="columns", lines=False))

For the starting points, the x coordinates were held constant, and the y and z coordinates varied by 0.65,.7,.75,.8,.85. The force was scaled by 0.7 , 0.8 , 0.9 , 1 , 1.1 , 1.2 , 1.3

There will be a live tracker of each trajectory, showing the step, time, velocity and position. Notice how in some cases the function runs indefinetly, with the velocity stagnating. This is an erroneous test case, and will need to be further examined. A timed kill switch has been added to end the function if it runs for too long. As the cell below runs, notice if any such erroneous test cases occur.

In [ ]:
y_coords = np.array([.65,.7,.75,.8,.85,])
z_coords = np.array([.65,.7,.75,.8,.85,])
x_coords = 0
#x_coords = np.array([0,.05,.1,.15,.2])
#y_coords = np.array([0,.05,.1,.15,.2])
#z_coords = np.array([0,.05,.1,.15,.2])
basefile = "./around1/new_trajectories_"
for i in np.array([0.7,0.8,0.9,1.0,1.1,1.2,1.3]):
    force_variation(x_coords,y_coords,z_coords,i,f"{basefile}{i}")

## Plotting Graphs
A displacement with respect to time graph will be plotted. With this, will be a force with respect to time graph, to show the force imparted on the particle as it traverses along. The example shown below is a random test case which works as expected.

In [ ]:
y_coords = np.array([.65,.7,.75,.8,.85])
z_coords = np.array([.65,.7,.75,.8,.85])
alltrajs = []
allforces = []
alldisplacements = []
allvelocities = []
times = []
for i in y_coords:
    for j in z_coords:
        traj = json.load(open(os.path.join("./111/new_trajectories_4", f'traj_{i}_{j}.json')))
        #traj = pd.read_json((os.path.join("./111/new_trajectories_4", f'traj_{i}_{j}.json')))
        displacement = list(traj['displacement'].values())
        velocity = list(traj['velocity'].values())
        time = list(traj['time'].values())
        times.append(time)
        alldisplacements.append(displacement)
        allvelocities.append(velocity)
        fun = traj_int.create_force_calculator_given_displacement([0,i,j],[4/np.sqrt(3), 4/np.sqrt(3), 4/np.sqrt(3)],1)
        forces = []
        for k in range(len(displacement)):
            force = fun(displacement[k],velocity[k])
            forces.append(force)
        allforces.append(forces)
            


However the following case is an erroneous test cases. One can see the displacement and the force with respect to time graphs. Here, we will have to troubleshoot the error to figure out why this test case specifically causes errors. 

In [ ]:
basefile = "./111/error_test_case_"
force_variation(0,[0.7],[0.7],4,f"{basefile}{4}")
traj = json.load(open(os.path.join("./111/error_test_case_4", f'traj_{0.7}_{0.7}.json')))
displacement = list(traj['displacement'].values())
velocity = list(traj['velocity'].values())
time = list(traj['time'].values())
fun = traj_int.create_force_calculator_given_displacement([0,i,j],[4/np.sqrt(3), 4/np.sqrt(3), 4/np.sqrt(3)],1)
forces = []
for i in range(len(displacement)):
    force = fun(displacement[i],velocity[i])
    forces.append(force)
